<a href="https://colab.research.google.com/github/JSJeong-me/LiteLLM-OnDeive-App/blob/main/004-RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb

데이터 준비

In [2]:
documents = [
    "Ollama Llama3 is a powerful language model designed for various natural language processing tasks.",
    "Retrieval-Augmented Generation (RAG) combines the strengths of information retrieval and natural language generation.",
    "The integration of retrieval with generation allows for more accurate and contextually relevant responses.",
    "Ollama Llama3 can be used in various applications, including chatbots, question answering, and more."
]

검색(리트리벌) 단계 - Chroma DB 사용

In [3]:
from transformers import AutoTokenizer, AutoModel
import chromadb
import numpy as np

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = AutoModel.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# 문서 임베딩 생성
def embed(documents):
    inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.pooler_output.detach().numpy()  # pooler_output 사용
    return embeddings

document_embeddings = embed(documents)

# Chroma DB 클라이언트 생성 및 문서 추가
client = chromadb.Client()

# 컬렉션 생성
collection = client.create_collection(name="document_collection")

# 문서 추가
for i, (doc, embedding) in enumerate(zip(documents, document_embeddings)):
    collection.add(
        ids=[f"doc_{i}"],  # 문서 ID
        embeddings=[embedding.tolist()],  # 임베딩 리스트
        metadatas=[{"text": doc}]  # 메타데이터 리스트
    )

# 검색 함수
def search(query, k=1):
    query_embedding = embed([query])[0]
    results = collection.query(query_embeddings=[query_embedding.tolist()], n_results=k)
    return [result["text"] for result in results["metadatas"][0]]



tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base and are newly initialized: ['bert_model.embeddings.LayerNorm.bias', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.word_embeddings.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.0.attention.output.dense.bias', 'bert_model.encoder.layer.0.attention.output.dense.weight', 'bert_model.encoder.layer.0.attention.self.key.bias', 'bert_model.encoder.layer.0.attention.self.key.weight', 'bert_model.encoder.layer.0.attention.self.query.bias', 'bert_model.encoder.layer.0.attention.self.query.weight', 'bert_model.encoder.layer.0.attention.self.value.bias', 'bert_model.encoder.layer.0.attention.self.value.weight', 'bert_model.encoder.layer.0.i

생성(제너레이션) 단계
Ollama Llama3를 사용하여 검색된 문서에 기반한 답변을 생성합니다. 이전 예제의 생성 단계와 동일합니다.

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 생성 모델과 토크나이저 로드
gen_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
gen_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")

# 검색된 문서로부터 답변 생성
def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    inputs = gen_tokenizer(input_text, return_tensors="pt")
    outputs = gen_model.generate(**inputs)
    answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

전체 RAG 파이프라인
질문을 입력받아 검색과 생성을 통해 답변을 출력하는 전체 파이프라인을 작성합니다.

In [5]:
def rag_pipeline(question):
    # 문서 검색
    relevant_documents = search(question, k=1)
    relevant_document = relevant_documents[0] if relevant_documents else "No relevant document found."

    # 답변 생성
    answer = generate_answer(question, relevant_document)
    return answer

# 실습 질문
question = "What is Retrieval-Augmented Generation?"
answer = rag_pipeline(question)
print(f"Question: {question}\nAnswer: {answer}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Question: What is Retrieval-Augmented Generation?
Answer: question: What is Retrieval-Augmented Generation? context: Retri
